# Inaugural Project

Imports and set magics:

In [8]:
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
plt.style.use("seaborn")

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import inauguralproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

We consider a consumer solving the following maximization problem

$$ 
\begin{aligned}
U^* = U(c^*,l^*) & = \text{arg}\max_{c,l}\log(c)-\nu\frac{l^{1+\frac{1}{\epsilon}}}{1 + \frac{1}{\epsilon}}\\
& \text{s.t.}\\
x & = m+wl-[\tau_{0}wl + \tau_{1} \max \{wl-\kappa, 0\}]\\
c &\in [0, x]\\
l &\in [0, 1],
\end{aligned} 
$$

where $c$ is consumption, $l$ is labor supply, $m$ is cash-on-hand, $w$ is the wage rate, $\tau_0$ is the
standard labor income tax, $\tau_0$ is the top bracket labor income tax, $\kappa$ is the cut-off for the top labor income bracket, $x$ is total resources, $\nu$ scales the disutility of labor, and $\epsilon$ is the Frisch elasticity of labor supply.
Utility is monotonically increasing in consumption, which implies that

$$
\begin{align}
c^* & = x
\end{align}
$$

Thus, inserting for c and subsequently for x yields the following univariate maximisation problem with only one constraint: 

$$ 
\begin{aligned}
U^* & = \text{arg}\max_{l}\log(m+wl-[\tau_{0}wl + \tau_{1} \max \{wl-\kappa, 0\}])-\nu\frac{l^{1+\frac{1}{\epsilon}}}{1 + \frac{1}{\epsilon}}\\
& \text{s.t.}\\
l &\in [0, 1]
\end{aligned} 
$$

The above utility function is defined in the module.

In [7]:
# code for solving the model (remember documentation and comments)

# b. set parameters
l = float()
m = 1
nu = 10
eps = 0.3
tau0 = 0.4
tau1 = 0.1
kappa = 0.4
w = 1


# c. set objective function
obj = lambda l: -inauguralproject.u_func(l,eps,kappa,nu,m,tau0,tau1,w)

# d. set constraints
con = lambda l: l
constraints = ({'type':'ineq','fun':con})

# e. call solver and derive level of consumption
initial_guess = 0
sol = optimize.minimize(obj,initial_guess,method='SLSQP',constraints=constraints)

c = m + w*l - (tau0*w*l + tau1*np.max(w*l - kappa,0))


# f. print solution
print(sol.message)
print(f'The agent will chooose l^star = {float(sol.x):.4f}, which implies c^star = {c:.4f} and U^star = {-sol.fun:.4f}')

Optimization terminated successfully.
The agent will chooose l^star = 0.3826, which implies c^star = 1.0400 and U^star = 0.1722


# Question 2

ADD ANSWER.

In [ ]:
w = np.linspace(0.5,1.5,num=100)



# Question 3

ADD ANSWER.

In [ ]:
# code

# Question 4

ADD ANSWER.

In [ ]:
# code

# Question 5

ADD ANSWER.

In [ ]:
# code

# Conclusion

ADD CONCISE CONLUSION.